In [4]:
# Probar con arbol con poda
# Probar con arbol sin poda pero corto
# Probar voting
# Probar varios KFolds en el de stacking

#Agregamos librerias a utilizar
import keras as keras
import pandas as pd
import xgboost as xgb
# import sklearn as sk
import numpy as np
from datetime import datetime

#Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt

#modelos y métricas
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split, cross_validate
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import StackingRegressor,RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
pd.set_option('display.max_columns', 40)

# import warnings
# warnings.simplefilter("ignore")

In [5]:
# Generamos un dataframe del dataset de entrenamiento
df =  pd.read_csv('train.csv')
df_final = df.drop(["shutdown", "mini_shutdown", "frankfurt_shutdown", "precipitation", "snow", "blackout", "mov_change", 'id'], axis = 1)
df_final['date'] = pd.to_datetime(df_final['date'])
df_final = df_final.sort_values(by = "date")
df_final_label = df_final.orders
df_final = df_final.drop(["orders"], axis=1)

def get_season(fecha):
    if fecha in ['December', 'January', 'February']:
        return 'Invierno'
    elif fecha in ['March', 'April', 'May']:
        return 'Primavera'
    elif fecha in ['June', 'July', 'August']:
        return 'Verano'
    elif fecha in ['September', 'October', 'November']:
        return 'Otoño'

# Separando el DataFrame en 80% y 20%
df_final_train, df_final_validation, df_final_train_label, df_final_validation_label = train_test_split(df_final,
                                                    df_final_label, 
                                                    test_size=0.2,
                                                    shuffle=False)
warehouses = df_final_train.warehouse
# mean user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].mean()
means_ua1= df_usact1.to_dict()

PROMEDIO_UA1 = df_usact1.mean()

# max user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].max()
max_ua1= df_usact1.to_dict()

# min user activity 1
df_usact1 = df_final_train[["user_activity_1", 'warehouse']]
df_usact1 = df_usact1.groupby("warehouse")["user_activity_1"].min()
min_ua1= df_usact1.to_dict()
# user activity 2
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].mean()
PROMEDIO_UA2 = df_usact1.mean()
means_ua2= df_usact2.to_dict()

# max user activity 1
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].max()
max_ua2= df_usact2.to_dict()

# min user activity 1
df_usact2 = df_final_train[["user_activity_2", 'warehouse']]
df_usact2 = df_usact2.groupby("warehouse")["user_activity_2"].min()
min_ua2= df_usact2.to_dict()
df_mean_encoding_warehouse = pd.concat([df_final_train, df_final_train_label], axis = 1)[["orders", 'warehouse']]
df_mean_encoding_warehouse = df_mean_encoding_warehouse.groupby("warehouse")["orders"].mean()
PROMEDIO_OPW = df_mean_encoding_warehouse.mean()
means_orders_per_warehouse= df_mean_encoding_warehouse.to_dict()
df_final_train['year_month_warehouse'] = df_final_train['date'].dt.month.astype(str) + '_' + df_final_train['warehouse']
df_mean_encoding_year_month_warehouse = pd.concat([df_final_train, df_final_train_label], axis = 1)[["orders", 'year_month_warehouse']]
df_mean_encoding_year_month_warehouse = df_mean_encoding_year_month_warehouse.groupby('year_month_warehouse')["orders"].mean()
PROMEDIO_YMW = df_mean_encoding_year_month_warehouse.mean()
means_y_m_w= df_mean_encoding_year_month_warehouse.to_dict()
#Features relacionadas con la user activity
df_final_train["mean_user_activity_1"] = df_final_train.warehouse.apply(lambda x: means_ua1.get(x, PROMEDIO_UA1))
df_final_train["mean_user_activity_2"] = df_final_train.warehouse.apply(lambda x: means_ua2.get(x, PROMEDIO_UA2))
df_final_train["max_user_activity_1"] = df_final_train.warehouse.apply(lambda x: max_ua1.get(x, PROMEDIO_UA1))
df_final_train["max_user_activity_2"] = df_final_train.warehouse.apply(lambda x: max_ua2.get(x, PROMEDIO_UA2))
df_final_train["min_user_activity_1"] = df_final_train.warehouse.apply(lambda x: min_ua1.get(x, PROMEDIO_UA1))
df_final_train["min_user_activity_2"] = df_final_train.warehouse.apply(lambda x: min_ua2.get(x, PROMEDIO_UA2))
df_final_train["product_ua"] = df_final_train["mean_user_activity_1"] * df_final_train["mean_user_activity_2"]


#Mean encoding
df_final_train["mean_orders_per_warehouse"] = df_final_train.warehouse.apply(lambda x: means_orders_per_warehouse.get(x, PROMEDIO_OPW))
df_final_train['year_month_warehouse'] = df_final_train['date'].dt.month.astype(str) + '_' + df_final_train['warehouse']
df_final_train["means_year_month_warehouse"] = df_final_train.year_month_warehouse.apply(lambda x: means_y_m_w.get(x, PROMEDIO_YMW))

city_to_country = {
    'Munich_1': 'Germany',
    'Frankfurt_1': 'Germany',
    'Budapest_1': 'Hungary',
    'Brno_1': 'Czech Republic',
    'Prague_1': 'Czech Republic',
    'Prague_2': 'Czech Republic',
    'Prague_3': 'Czech Republic'
}

#Countries and warehouses
df_final_train['country'] = df_final_train['warehouse'].map(city_to_country)
df_final_train = pd.concat([df_final_train.drop(["country"], axis=1), pd.get_dummies(df_final_train.country).astype('int').add_prefix('country_')], axis=1)
df_final_train = pd.concat([df_final_train.drop(["warehouse"], axis=1), pd.get_dummies(df_final_train.warehouse).astype('int').add_prefix('warehouse_')], axis=1)
df_final_train = df_final_train.drop(["warehouse_Budapest_1", "warehouse_Prague_2", "warehouse_Prague_3"], axis=1)
df_final_train['hot_season'] = df_final_train.date.apply(lambda x: 1 if 2 < x.month < 9 else 0)

#Time
df_final_train['year'] = df_final_train['date'].dt.year
df_final_train['month'] = df_final_train['date'].dt.month
df_final_train['week'] = df_final_train['date'].dt.isocalendar().week
df_final_train['day'] = df_final_train['date'].dt.day
df_final_train['day_of_week'] = df_final_train['date'].dt.dayofweek #numero
# df_final_train['day_of_week'] = df_final_train['date'].dt.day_name() #cadena
df_final_train['year_sin'] = np.sin(2 * np.pi * df_final_train['year'])
#df_final_train['year_cos'] = np.cos(2 * np.pi * df_final_train['year'])
#df_final_train['month_sin'] = np.sin(2 * np.pi * df_final_train['month'] / 12) 
df_final_train['month_cos'] = np.cos(2 * np.pi * df_final_train['month'] / 12)
#df_final_train['week_sin'] = np.sin(2 * np.pi * df_final_train['week'] / 53)
df_final_train['week_cos'] = np.cos(2 * np.pi * df_final_train['week'] / 53)
#df_final_train['day_sin'] = np.sin(2 * np.pi * df_final_train['day'] / 31)
#df_final_train['day_cos'] = np.cos(2 * np.pi * df_final_train['day'] / 31)
#df_final_train['day_of_week_sin'] = np.sin(2 * np.pi * df_final_train['day_of_week'] / 7)
#df_final_train['day_of_week_cos'] = np.cos(2 * np.pi * df_final_train['day_of_week'] / 7)
df_final_train["month_name"] = df_final_train.date.dt.month_name()
df_final_train['season'] = df_final_train.month_name.apply(get_season)
df_final_train = pd.concat([df_final_train, pd.get_dummies(df_final_train.season).astype('int').add_prefix('Season_')], axis=1)
df_final_train['group']=(df_final_train['year']-2020)*48+df_final_train['month']*4+df_final_train['day']//7

#drops
df_final_train = df_final_train.drop(['date', 'year', 'month', 'week', 'day', 'day_of_week', 'month_name', 'season', 'holiday_name', 'year_month_warehouse', 'user_activity_1', 'user_activity_2'], axis = 1)
df_final_train = df_final_train.reset_index().drop(["index"], axis = 1)

warehouses_validation = df_final_validation.warehouse
#Features relacionadas con la user activity
df_final_validation["mean_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: means_ua1.get(x, PROMEDIO_UA1))
df_final_validation["mean_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: means_ua2.get(x, PROMEDIO_UA2))
df_final_validation["max_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: max_ua1.get(x, PROMEDIO_UA1))
df_final_validation["max_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: max_ua2.get(x, PROMEDIO_UA2))
df_final_validation["min_user_activity_1"] = df_final_validation.warehouse.apply(lambda x: min_ua1.get(x, PROMEDIO_UA1))
df_final_validation["min_user_activity_2"] = df_final_validation.warehouse.apply(lambda x: min_ua2.get(x, PROMEDIO_UA2))
df_final_validation["product_ua"] = df_final_validation["mean_user_activity_1"] * df_final_validation["mean_user_activity_2"]


#Mean encoding
df_final_validation["mean_orders_per_warehouse"] = df_final_validation.warehouse.apply(lambda x: means_orders_per_warehouse.get(x, PROMEDIO_OPW))
df_final_validation['year_month_warehouse'] = df_final_validation['date'].dt.month.astype(str) + '_' + df_final_validation['warehouse']
df_final_validation["means_year_month_warehouse"] = df_final_validation.year_month_warehouse.apply(lambda x: means_y_m_w.get(x, PROMEDIO_YMW))

city_to_country = {
    'Munich_1': 'Germany',
    'Frankfurt_1': 'Germany',
    'Budapest_1': 'Hungary',
    'Brno_1': 'Czech Republic',
    'Prague_1': 'Czech Republic',
    'Prague_2': 'Czech Republic',
    'Prague_3': 'Czech Republic'
}

#Countries and warehouses
df_final_validation['country'] = df_final_validation['warehouse'].map(city_to_country)
df_final_validation = pd.concat([df_final_validation.drop(["country"], axis=1), pd.get_dummies(df_final_validation.country).astype('int').add_prefix('country_')], axis=1)
df_final_validation = pd.concat([df_final_validation.drop(["warehouse"], axis=1), pd.get_dummies(df_final_validation.warehouse).astype('int').add_prefix('warehouse_')], axis=1)
df_final_validation = df_final_validation.drop(["warehouse_Budapest_1", "warehouse_Prague_2", "warehouse_Prague_3"], axis=1)
df_final_validation['hot_season'] = df_final_validation.date.apply(lambda x: 1 if 2 < x.month < 9 else 0)

#Time
df_final_validation['year'] = df_final_validation['date'].dt.year
df_final_validation['month'] = df_final_validation['date'].dt.month
df_final_validation['week'] = df_final_validation['date'].dt.isocalendar().week
df_final_validation['day'] = df_final_validation['date'].dt.day
df_final_validation['day_of_week'] = df_final_validation['date'].dt.dayofweek #numero
# df_final_validation['day_of_week'] = df_final_validation['date'].dt.day_name() #cadena
df_final_validation['year_sin'] = np.sin(2 * np.pi * df_final_validation['year'])
#df_final_validation['year_cos'] = np.cos(2 * np.pi * df_final_validation['year'])
#df_final_validation['month_sin'] = np.sin(2 * np.pi * df_final_validation['month'] / 12) 
df_final_validation['month_cos'] = np.cos(2 * np.pi * df_final_validation['month'] / 12)
#df_final_validation['week_sin'] = np.sin(2 * np.pi * df_final_validation['week'] / 53)
df_final_validation['week_cos'] = np.cos(2 * np.pi * df_final_validation['week'] / 53)
#df_final_validation['day_sin'] = np.sin(2 * np.pi * df_final_validation['day'] / 31)
#df_final_validation['day_cos'] = np.cos(2 * np.pi * df_final_validation['day'] / 31)
#df_final_validation['day_of_week_sin'] = np.sin(2 * np.pi * df_final_validation['day_of_week'] / 7)
#df_final_validation['day_of_week_cos'] = np.cos(2 * np.pi * df_final_validation['day_of_week'] / 7)
df_final_validation["month_name"] = df_final_validation.date.dt.month_name()
df_final_validation['season'] = df_final_validation.month_name.apply(get_season)
df_final_validation = pd.concat([df_final_validation, pd.get_dummies(df_final_validation.season).astype('int').add_prefix('Season_')], axis=1)
df_final_validation['group']=(df_final_validation['year']-2020)*48+df_final_validation['month']*4+df_final_validation['day']//7

#drops
df_final_validation = df_final_validation.drop(['date', 'year', 'month', 'week', 'day', 'day_of_week', 'month_name', 'season', 'holiday_name', 'year_month_warehouse', 'user_activity_1', 'user_activity_2'], axis = 1)
df_final_validation = df_final_validation.reset_index().drop(["index"], axis = 1)

df_for_corr = pd.concat([df_final_train, df_final_train_label], axis = 1).corr()
features_orders  = df_for_corr.sort_values(by="orders", ascending=False).orders
best_features = list(features_orders[features_orders > 0.1].index)
best_features.pop(0)
# Reordenar las columnas del DataFrame según el orden deseado
df_final_train = df_final_train[best_features]
df_final_validation = df_final_validation[best_features]

In [6]:
df_final_train

,year_sin,week_cos,month_cos,Season_Invierno,means_year_month_warehouse,min_user_activity_2,mean_orders_per_warehouse,mean_user_activity_2,max_user_activity_2,country_Czech Republic,product_ua,min_user_activity_1,mean_user_activity_1
0,-8.073961e-13,0.889657,1.0,1,8767.872093,30403.0,8261.953704,35952.602881,41952.0,1,6.017289e+07,1564.0,1673.672840
1,-8.073961e-13,0.889657,1.0,1,5268.209302,17799.0,4982.338477,20996.281893,24401.0,1,2.737446e+07,1248.0,1303.776749
2,-8.073961e-13,0.889657,1.0,1,4864.523256,16254.0,4576.633094,18871.905447,21411.0,1,1.804961e+07,914.0,956.427544
3,-8.073961e-13,0.889657,1.0,1,5482.445783,17478.0,5362.533546,21435.602769,24488.0,0,6.409022e+07,2921.0,2989.895634
4,-8.073961e-13,0.889657,1.0,1,7309.081395,25037.0,7009.977366,29416.394033,34040.0,1,6.837638e+07,2245.0,2324.431070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,-6.447061e-13,-0.861044,-0.5,0,7320.764706,30403.0,8261.953704,35952.602881,41952.0,1,6.017289e+07,1564.0,1673.672840
5868,-6.447061e-13,-0.861044,-0.5,0,4983.166667,17478.0,5362.533546,21435.602769,24488.0,0,6.409022e+07,2921.0,2989.895634
5869,-6.447061e-13,-0.861044,-0.5,0,4504.602941,17799.0,4982.338477,20996.281893,24401.0,1,2.737446e+07,1248.0,1303.776749
5870,-6.447061e-13,-0.794854,-0.5,0,4095.637681,16254.0,4576.633094,18871.905447,21411.0,1,1.804961e+07,914.0,956.427544


In [7]:
df_final_train = df_final_train.drop(["country_Czech Republic"], axis = 1)
df_final_validation = df_final_validation.drop(["country_Czech Republic"], axis = 1)
# Escalar los datos
unique_warehouses = list(warehouses.unique())
models_warehouses = {}
scaler = MinMaxScaler()
warehouses = warehouses.reset_index().drop(["index"], axis = 1)
orders = df_final_train_label.reset_index().drop(["index"], axis = 1) 
df_final_train['warehouse'] = warehouses
df_final_train["orders"] = orders



In [9]:
warehouses.value_counts()

warehouse  
Prague_3       973
Prague_1       972
Brno_1         972
Prague_2       972
Budapest_1     939
Munich_1       602
Frankfurt_1    442
Name: count, dtype: int64

In [8]:
df_final_train.count()

year_sin                      5872
week_cos                      5872
month_cos                     5872
Season_Invierno               5872
means_year_month_warehouse    5872
min_user_activity_2           5872
mean_orders_per_warehouse     5872
mean_user_activity_2          5872
max_user_activity_2           5872
product_ua                    5872
min_user_activity_1           5872
mean_user_activity_1          5872
warehouse                     5872
orders                        5872
dtype: int64

In [10]:
for warehouse in unique_warehouses:
   df_with_unique_warehouse = df_final_train[df_final_train.warehouse == warehouse].drop(['warehouse'], axis = 1)
   df_with_unique_warehouse_label = df_with_unique_warehouse.orders
   df_with_unique_warehouse = df_with_unique_warehouse.drop(['orders'], axis = 1)

   df_train_scaled_with_unique_warehouse = scaler.fit_transform(df_with_unique_warehouse)
   # Crear el modelo de red neuronal
   model_nn = Sequential()
   model_nn.add(Dense(64, input_dim=df_train_scaled_with_unique_warehouse.shape[1], activation='relu'))
   model_nn.add(Dense(32, activation='relu'))
   model_nn.add(Dense(1))

   # Compilar el modelo
   model_nn.compile(loss='mean_absolute_percentage_error', optimizer='adam')

   # Entrenar el modelo
   history = model_nn.fit(df_train_scaled_with_unique_warehouse, df_with_unique_warehouse_label, epochs=100, batch_size=10, verbose=2, validation_split=0.2)
   models_warehouses[warehouse] = model_nn
   
   results = pd.Series(dtype='float64')

def predict_by_warehouse(row):
    warehouse = warehouses_validation.iloc[row.name]
    
    model = models_warehouses[warehouse]

    scaled_row = scaler.fit_transform(row.values.reshape(1, -1))
    result = model.predict(scaled_row)

    results.loc[len(results)] = result[0]

df_final_validation.apply(predict_by_warehouse, axis=1)


/home/lminervino18/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
78/78 - 2s - 23ms/step - loss: 99.9843 - val_loss: 99.9610
Epoch 2/100
78/78 - 0s - 4ms/step - loss: 99.8260 - val_loss: 99.6831
Epoch 3/100
78/78 - 0s - 4ms/step - loss: 99.1357 - val_loss: 98.7594
Epoch 4/100
78/78 - 0s - 2ms/step - loss: 97.3716 - val_loss: 96.7288
Epoch 5/100
78/78 - 0s - 1ms/step - loss: 93.9875 - val_loss: 93.1944
Epoch 6/100
78/78 - 0s - 1ms/step - loss: 88.5185 - val_loss: 87.8208
Epoch 7/100
78/78 - 0s - 1ms/step - loss: 80.5912 - val_loss: 80.2821
Epoch 8/100
78/78 - 0s - 1ms/step - loss: 69.8534 - val_loss: 70.3679
Epoch 9/100
78/78 - 0s - 1ms/step - loss: 56.4743 - val_loss: 58.0211
Epoch 10/100
78/78 - 0s - 1ms/step - loss: 42.1331 - val_loss: 45.5254
Epoch 11/100
78/78 - 0s - 2ms/step - loss: 33.6498 - val_loss: 37.6576
Epoch 12/100
78/78 - 0s - 3ms/step - loss: 29.6386 - val_loss: 34.1182
Epoch 13/100
78/78 - 0s - 5ms/step - loss: 28.1938 - val_loss: 32.2475
Epoch 14/100
78/78 - 0s - 3ms/step - loss: 27.4446 - val_loss: 31.1400
Epoch 15/100
7

/home/lminervino18/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 - 1s - 14ms/step - loss: 99.9577 - val_loss: 99.8923
Epoch 2/100
78/78 - 0s - 2ms/step - loss: 99.5519 - val_loss: 99.1950
Epoch 3/100
78/78 - 0s - 2ms/step - loss: 97.9298 - val_loss: 97.0774
Epoch 4/100
78/78 - 0s - 2ms/step - loss: 94.0821 - val_loss: 92.7563
Epoch 5/100
78/78 - 0s - 4ms/step - loss: 87.0607 - val_loss: 85.4863
Epoch 6/100
78/78 - 0s - 3ms/step - loss: 75.8922 - val_loss: 74.5306
Epoch 7/100
78/78 - 0s - 2ms/step - loss: 59.9857 - val_loss: 59.3728
Epoch 8/100
78/78 - 0s - 2ms/step - loss: 42.0187 - val_loss: 43.9606
Epoch 9/100
78/78 - 0s - 2ms/step - loss: 32.5548 - val_loss: 35.5041
Epoch 10/100
78/78 - 0s - 2ms/step - loss: 28.8557 - val_loss: 31.9193
Epoch 11/100
78/78 - 0s - 3ms/step - loss: 27.7420 - val_loss: 30.5926
Epoch 12/100
78/78 - 0s - 3ms/step - loss: 26.9587 - val_loss: 29.2559
Epoch 13/100
78/78 - 0s - 2ms/step - loss: 26.1893 - val_loss: 28.1953
Epoch 14/100
78/78 - 0s - 2ms/step - loss: 25.4848 - val_loss: 27.2313
Epoch 15/100
78/78 - 0s - 

/home/lminervino18/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/home/lminervino18/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


49/49 - 2s - 31ms/step - loss: 99.9736 - val_loss: 99.9530
Epoch 2/100
49/49 - 0s - 4ms/step - loss: 99.8025 - val_loss: 99.7084
Epoch 3/100
49/49 - 0s - 3ms/step - loss: 99.0925 - val_loss: 98.9575
Epoch 4/100
49/49 - 0s - 3ms/step - loss: 97.3544 - val_loss: 97.2992
Epoch 5/100
49/49 - 0s - 3ms/step - loss: 93.8785 - val_loss: 94.3146
Epoch 6/100
49/49 - 0s - 3ms/step - loss: 88.0261 - val_loss: 89.6019
Epoch 7/100
49/49 - 0s - 6ms/step - loss: 79.1415 - val_loss: 82.8715
Epoch 8/100
49/49 - 0s - 3ms/step - loss: 66.7843 - val_loss: 73.6851
Epoch 9/100
49/49 - 0s - 7ms/step - loss: 51.9526 - val_loss: 63.3682
Epoch 10/100
49/49 - 0s - 5ms/step - loss: 42.1856 - val_loss: 56.0076
Epoch 11/100
49/49 - 0s - 3ms/step - loss: 38.8473 - val_loss: 52.8113
Epoch 12/100
49/49 - 0s - 3ms/step - loss: 37.4239 - val_loss: 50.7194
Epoch 13/100
49/49 - 0s - 3ms/step - loss: 36.4341 - val_loss: 49.6963
Epoch 14/100
49/49 - 0s - 3ms/step - loss: 35.4584 - val_loss: 48.1990
Epoch 15/100
49/49 - 0s - 

/home/lminervino18/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


36/36 - 1s - 32ms/step - loss: 99.9498 - val_loss: 99.8924
Epoch 2/100
36/36 - 0s - 6ms/step - loss: 99.7396 - val_loss: 99.6008
Epoch 3/100
36/36 - 0s - 8ms/step - loss: 99.0673 - val_loss: 98.7332
Epoch 4/100
36/36 - 0s - 8ms/step - loss: 97.3874 - val_loss: 96.7822
Epoch 5/100
36/36 - 0s - 5ms/step - loss: 94.0507 - val_loss: 93.2807
Epoch 6/100
36/36 - 0s - 4ms/step - loss: 88.5937 - val_loss: 87.6593
Epoch 7/100
36/36 - 0s - 3ms/step - loss: 80.3066 - val_loss: 79.6440
Epoch 8/100
36/36 - 0s - 4ms/step - loss: 68.7256 - val_loss: 68.7701
Epoch 9/100
36/36 - 0s - 3ms/step - loss: 53.3145 - val_loss: 54.5957
Epoch 10/100
36/36 - 0s - 4ms/step - loss: 38.6553 - val_loss: 41.6281
Epoch 11/100
36/36 - 0s - 4ms/step - loss: 32.5041 - val_loss: 32.9857
Epoch 12/100
36/36 - 0s - 4ms/step - loss: 28.8546 - val_loss: 28.0860
Epoch 13/100
36/36 - 0s - 3ms/step - loss: 26.6976 - val_loss: 23.0725
Epoch 14/100
36/36 - 0s - 6ms/step - loss: 25.0674 - val_loss: 19.4984
Epoch 15/100
36/36 - 0s - 

0       None
1       None
2       None
3       None
4       None
        ... 
1463    None
1464    None
1465    None
1466    None
1467    None
Length: 1468, dtype: object

In [11]:
results

0       [4507.2676]
1        [6359.823]
2       [1025.2122]
3         [7317.15]
4        [5105.222]
           ...     
1463     [6359.823]
1464    [4073.9001]
1465      [7317.15]
1466    [4507.2676]
1467    [1025.2122]
Length: 1468, dtype: object

In [12]:
df_final_validation_label

3358     5020.0
2165     7319.0
5999     1565.0
972      8455.0
7125     5607.0
         ...   
2385    10777.0
4771     6408.0
1192    11917.0
3578     7140.0
6185     1916.0
Name: orders, Length: 1468, dtype: float64

In [13]:
mape = mean_absolute_percentage_error(df_final_validation_label, results)
print(f'Test MAPE: {mape}')

Test MAPE: 0.29471694675517035
